In [18]:
#1.1 Replicating the Regression Table (9 points)
!pip install ISLP
from ISLP import load_data
import pandas as pd

url = "https://raw.githubusercontent.com/selva86/datasets/master/Advertising.csv"
Advertising = pd.read_csv(url)
display(Advertising.head())


# Fit the model and display the summary
import statsmodels.api as sm
X = Advertising[['TV', 'radio', 'newspaper']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = Advertising['sales']
model = sm.OLS(y, X).fit()
#print(model.summary())

# Create a regression table
import numpy as np

table = pd.DataFrame({
    "Coefficient": model.params,
    "Std. error": model.bse,
    "t-statistic": model.tvalues,
    "p-value": model.pvalues
})

table["Coefficient"] = table["Coefficient"].round(3)
table["Std. error"]  = table["Std. error"].round(4)
table["t-statistic"] = table["t-statistic"].round(2)

table["p-value"] = table["p-value"].apply(lambda p: "<0.0001" if p < 0.0001 else round(p,4))

table = table.rename(index={'const': 'Intercept'})

display(table)

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


,Coefficient,Std. error,t-statistic,p-value
Intercept,2.939,0.3119,9.42,<0.0001
TV,0.046,0.0014,32.81,<0.0001
radio,0.189,0.0086,21.89,<0.0001
newspaper,-0.001,0.0059,-0.18,0.8599


### 1.2 Hypotheses for the p-values (7 points)

For each predictor, the null hypothesis states that its coefficient is equal to zero.  
In the context of the Advertising data, this means:

- **TV**: The null hypothesis is that TV advertising has no effect on sales, after controlling for radio and newspaper.  
- **Radio**: The null hypothesis is that radio advertising has no effect on sales, after controlling for TV and newspaper.  
- **Newspaper**: The null hypothesis is that newspaper advertising has no effect on sales, after controlling for TV and radio.  


### 1.3 Interpreting the Results (7 points)

Based on the p-values from the regression output:

- **TV**: The p-value is very small (<0.0001), so TV advertising has a strong, statistically significant positive relationship with sales.  
- **Radio**: The p-value is also very small (<0.0001), so radio advertising likewise shows a statistically significant positive relationship with sales.  
- **Newspaper**: The p-value is large (about 0.86), so there is no statistical evidence that newspaper advertising is related to sales.  

**Conclusion:** TV and radio advertising both significantly increase sales, while newspaper advertising does not appear to have a meaningful effect.


### Question 2 (10 points)

**K-Nearest Neighbors (KNN) Overview**  
The K-Nearest Neighbors (KNN) method is a non-parametric, instance-based learning approach.  
When predicting the outcome for a new observation, the algorithm looks for the **K training observations that are closest** to it in the feature space, where “closeness” is usually measured by Euclidean distance. The new observation’s prediction is then based on these neighbors.

**KNN Classifier**  
- Used for classification problems (predicting categories).  
- The algorithm finds the K nearest neighbors and assigns the most common class among them (majority vote) to the new observation.  
- Example: If 3 of the 5 nearest neighbors are “Yes” and 2 are “No,” the new observation is classified as “Yes.”

**KNN Regression**  
- Used for regression problems (predicting continuous values).  
- The algorithm finds the K nearest neighbors and takes the **average or weighted average** of their target values as the prediction.  
- Example: If the 3 nearest neighbors have house prices of 100k, 110k, and 120k, the predicted price is about 110k.

**Key Differences**  
| Aspect        | KNN Classifier (Classification)     | KNN Regression (Regression)   |
|---------------|-------------------------------------|--------------------------------|
| Output        | A class label (e.g., Yes/No)        | A continuous value (e.g., price) |
| Decision rule | Majority vote among neighbors       | Average or weighted average of neighbors’ values |

**Role of Distance**  
“Distance” means how close two points are in terms of their features.  
The most common metric is **Euclidean distance**, which is the straight-line distance between two points in the feature space. If two observations have very similar feature values, their distance is small, and they are considered neighbors.  

In [19]:
# === Q3 Setup: helpers & data generators ===
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Design matrices
def dm_linear(X):
    #Linear model with intercept
    return sm.add_constant(pd.DataFrame({'X': X}))

def dm_cubic(X):
    #Cubic model with intercept
    return sm.add_constant(pd.DataFrame({'X': X, 'X2': X**2, 'X3': X**3}))

# Fit and compute RSS
def fit_and_rss(X, y, dm_fn):
    #Fit OLS with a given design-matrix function and return (RSS, fitted_results).
    Xdm = dm_fn(X)
    res = sm.OLS(y, Xdm).fit()
    rss = float(np.sum(res.resid**2))
    return rss, res

# Simulators
def sim_linear_truth(n=100, beta0=2.0, beta1=3.0, noise_sd=1.0, rng=None):
    #Generate data from a linear truth: y = beta0 + beta1 * X + noise.
    rng = rng or np.random.default_rng()
    X = rng.normal(size=n)
    y = beta0 + beta1 * X + rng.normal(scale=noise_sd, size=n)
    return X, y

def sim_nonlinear_truth(n=100, beta0=2.0, beta1=1.0, quad=0.8, cubic=0.3, noise_sd=1.0, rng=None):
    #Generate data from a nonlinear truth: y = beta0 + beta1*X + quad*X^2 + cubic*X^3 + noise.
    rng = rng or np.random.default_rng()
    X = rng.normal(size=n)
    f = beta0 + beta1*X + quad*(X**2) + cubic*(X**3)
    y = f + rng.normal(scale=noise_sd, size=n)
    return X, y


In [20]:
# === Q3(a): Training RSS (linear truth, fixed seed) ===
rng = np.random.default_rng(2025)

# Generate training data under a linear truth
X_tr, y_tr = sim_linear_truth(n=100, beta0=2.0, beta1=3.0, noise_sd=1.0, rng=rng)

# Fit linear vs cubic on training data; compare training RSS
rss_lin_tr, res_lin = fit_and_rss(X_tr, y_tr, dm_linear)
rss_cub_tr, res_cub = fit_and_rss(X_tr, y_tr, dm_cubic)

print("Q3(a) — Training RSS (true linear):")
print(f"  Linear RSS (train): {rss_lin_tr:.3f}")
print(f"  Cubic  RSS (train): {rss_cub_tr:.3f}")
print("Conclusion: Cubic training RSS is no greater than Linear training RSS (nested model & OLS minimizes RSS).")


Q3(a) — Training RSS (true linear):
  Linear RSS (train): 113.029
  Cubic  RSS (train): 112.994
Conclusion: Cubic training RSS is no greater than Linear training RSS (nested model & OLS minimizes RSS).


**Q3 (a)**

The cubic model is more flexible because it includes the linear model as a special case (when β₂ = 0 and β₃ = 0, the cubic model reduces to the linear model). Since OLS always finds the coefficients that minimize the training RSS, the cubic regression searches over a larger set of possible fits. Therefore, the cubic training RSS will always be less than or equal to the linear training RSS.

In our simulation with a true linear relationship, the training RSS for the linear model was 113.029, while for the cubic model it was slightly lower at 112.994, confirming this theoretical result.

In [21]:
# === Q3(b): Test RSS (linear truth, average over many trials) ===
def avg_test_rss_linear_truth(n_train=100, n_test=1000, trials=200, seed=7):
    rng = np.random.default_rng(seed)
    lin_list, cub_list = [], []
    for _ in range(trials):
        # Training & test from same linear truth
        X_tr, y_tr = sim_linear_truth(n=n_train, beta0=2.0, beta1=3.0, noise_sd=1.0, rng=rng)
        X_te, y_te = sim_linear_truth(n=n_test,  beta0=2.0, beta1=3.0, noise_sd=1.0, rng=rng)

        # Fit on training
        _, res_lin = fit_and_rss(X_tr, y_tr, dm_linear)
        _, res_cub = fit_and_rss(X_tr, y_tr, dm_cubic)

        # Test RSS
        pred_lin = res_lin.predict(dm_linear(X_te))
        pred_cub = res_cub.predict(dm_cubic(X_te))
        lin_list.append(float(np.sum((y_te - pred_lin)**2)))
        cub_list.append(float(np.sum((y_te - pred_cub)**2)))

    return np.mean(lin_list), np.mean(cub_list)

lin_te_avg, cub_te_avg = avg_test_rss_linear_truth()
print("Q3(b) — Average Test RSS over trials (true linear):")
print(f"  Linear Test RSS (avg): {lin_te_avg:.2f}")
print(f"  Cubic  Test RSS (avg): {cub_te_avg:.2f}")
print("Conclusion: Linear has lower average test RSS (better generalization when the truth is linear).")


Q3(b) — Average Test RSS over trials (true linear):
  Linear Test RSS (avg): 1016.97
  Cubic  Test RSS (avg): 1055.15
Conclusion: Linear has lower average test RSS (better generalization when the truth is linear).


**Q3 (b)**

When we use test RSS instead of training RSS, the result can change. Even though the cubic model always achieves a lower (or equal) training RSS, it may overfit the noise in the training data. Since the true relationship is linear, the linear regression model should have a lower test RSS on average.

In our simulation, the average test RSS for the linear model was 1016.97, compared to 1055.15 for the cubic model. This confirms the expectation: while the cubic model can fit the training data slightly better, the linear model generalizes better to unseen test data when the underlying relationship is truly linear.

In [22]:
# === Q3(c): Training RSS (nonlinear truth, fixed seed) ===
rng = np.random.default_rng(1314)

# Generate training data under a nonlinear truth (you may tweak quad/cubic/noise_sd)
X_tr, y_tr = sim_nonlinear_truth(n=100, beta0=2.0, beta1=1.0, quad=0.8, cubic=0.3, noise_sd=1.0, rng=rng)

rss_lin_tr, _ = fit_and_rss(X_tr, y_tr, dm_linear)
rss_cub_tr, _ = fit_and_rss(X_tr, y_tr, dm_cubic)

print("Q3(c) — Training RSS (true nonlinear):")
print(f"  Linear RSS (train): {rss_lin_tr:.3f}")
print(f"  Cubic  RSS (train): {rss_cub_tr:.3f}")
print("Conclusion: Cubic training RSS is no greater than Linear training RSS; with nonlinearity, Cubic often strictly smaller.")

Q3(c) — Training RSS (true nonlinear):
  Linear RSS (train): 356.388
  Cubic  RSS (train): 95.794
Conclusion: Cubic training RSS is no greater than Linear training RSS; with nonlinearity, Cubic often strictly smaller.


**Q3 (c)**

This case differs from part (a) because the true relationship is not linear, so the linear model is misspecified. However, the cubic model still nests the linear model (if β₂ = β₃ = 0, the cubic reduces to the linear). Since OLS minimizes the residual sum of squares, the cubic regression cannot have a larger training RSS than the linear regression.

In our simulation, the training RSS for the linear model was 356.388, while the cubic model achieved a much smaller value of 95.794. This confirms the expectation: when the true function is nonlinear, the cubic regression typically attains a strictly lower training RSS than the linear regression.

In [23]:
# === Q3(d): Test RSS (nonlinear truth, average over many trials) ===
def avg_test_rss_nonlinear_truth(n_train=100, n_test=1000, trials=200,
                                 quad=0.8, cubic=0.3, noise_sd=1.0, seed=99):
    rng = np.random.default_rng(seed)
    lin_list, cub_list = [], []
    for _ in range(trials):
        # Training & test from the same nonlinear truth
        X_tr, y_tr = sim_nonlinear_truth(n=n_train, beta0=2.0, beta1=1.0,
                                         quad=quad, cubic=cubic, noise_sd=noise_sd, rng=rng)
        X_te, y_te = sim_nonlinear_truth(n=n_test,  beta0=2.0, beta1=1.0,
                                         quad=quad, cubic=cubic, noise_sd=noise_sd, rng=rng)

        # Fit on training
        _, res_lin = fit_and_rss(X_tr, y_tr, dm_linear)
        _, res_cub = fit_and_rss(X_tr, y_tr, dm_cubic)

        # Test RSS
        pred_lin = res_lin.predict(dm_linear(X_te))
        pred_cub = res_cub.predict(dm_cubic(X_te))
        lin_list.append(float(np.sum((y_te - pred_lin)**2)))
        cub_list.append(float(np.sum((y_te - pred_cub)**2)))

    return np.mean(lin_list), np.mean(cub_list)

lin_te_avg, cub_te_avg = avg_test_rss_nonlinear_truth(quad=0.8, cubic=0.3, noise_sd=1.0)
print("Q3(d) — Average Test RSS over trials (true nonlinear):")
print(f"  Linear Test RSS (avg): {lin_te_avg:.2f}")
print(f"  Cubic  Test RSS (avg): {cub_te_avg:.2f}")
print("Conclusion: It depends (bias–variance trade-off). Strong nonlinearity & moderate noise → cubic wins; otherwise linear may generalize better.")


Q3(d) — Average Test RSS over trials (true nonlinear):
  Linear Test RSS (avg): 2977.71
  Cubic  Test RSS (avg): 1056.17
Conclusion: It depends (bias–variance trade-off). Strong nonlinearity & moderate noise → cubic wins; otherwise linear may generalize better.


**Q3(d)**

When we talk about **test RSS**, we mean the model’s *generalization ability* — how well it predicts new data not used in training. A model that only memorizes training data (low training RSS) may still perform poorly on test data if it does not generalize well.  

**Bias–variance trade-off:**  
- **Bias**: error from assuming an overly simple model (e.g., forcing a straight line to fit a curve).  
- **Variance**: error from fitting training data too closely, including noise.  
- Linear models usually have higher bias but lower variance.  
- Cubic models usually have lower bias but higher variance (risk of overfitting).  

**Implications:**  
- If nonlinearity is strong and cubic terms capture it without overfitting, the cubic model will likely have lower test RSS.  
- If the relationship is only slightly nonlinear or noisy, the linear model may generalize better.  

**Simulation result:**  
- Linear test RSS (avg): 2977.71  
- Cubic test RSS (avg): 1056.17  

This confirms that under strong nonlinearity, the cubic model generalizes better than the linear one.

In [24]:
# Q4(a) — Simple linear regression of y ~ x (NO intercept)

import numpy as np
import pandas as pd
import statsmodels.api as sm

# 1) Generate data
rng = np.random.default_rng(1)
x = rng.normal(size=100)
y = 2 * x + rng.normal(size=100)

# 2) Fit OLS without intercept:
X = pd.DataFrame({'x': x})      
model = sm.OLS(y, X).fit()

# 3) Extract results
beta_hat = model.params['x']
se_beta = model.bse['x']
t_stat   = model.tvalues['x']
p_value  = model.pvalues['x']

print("Coefficient estimate (beta_hat):", beta_hat)
print("Standard error:", se_beta)
print("t-statistic:", t_stat)
print("p-value:", p_value)


Coefficient estimate (beta_hat): 1.976242377442051
Standard error: 0.11694837274226326
t-statistic: 16.898417063035104
p-value: 6.231546010056513e-31


### Q4(a) Report

- **Estimated coefficient (β̂):** 1.976  
- **Standard error:** 0.117  
- **t-statistic:** 16.90  
- **p-value:** 6.23 × 10⁻³¹  

**Comment:**  
The estimated coefficient is very close to the true value of 2, which makes sense because the data were generated from the model `y = 2x + ε`.  

The t-statistic is very large (≈ 16.9) and the p-value is essentially zero, so we strongly reject the null hypothesis `H0: β = 0`.  

This indicates that x is highly significant in explaining y when no intercept is included in the model.

In [25]:
# Q4(b) — Simple linear regression of x ~ y (NO intercept)
import statsmodels.api as sm
import pandas as pd

X = pd.DataFrame({'y': y})   # predictor
y_resp = x                   # response

# no intercept
model_b = sm.OLS(y_resp, X).fit()

beta_hat = model_b.params.iloc[0]
se       = model_b.bse.iloc[0]
t_stat   = model_b.tvalues.iloc[0]
p_val    = model_b.pvalues.iloc[0]


print("Coefficient estimate (beta_hat):", beta_hat)
print("Standard error:", se)
print("t-statistic:", t_stat)
print("p-value:", p_val)

Coefficient estimate (beta_hat): 0.37574368103348865
Standard error: 0.022235436587455238
t-statistic: 16.898417063035104
p-value: 6.231546010056513e-31


### Q4(b) Report

- **Estimated coefficient (β̂):** 0.376  
- **Standard error:** 0.022  
- **t-statistic:** 16.90  
- **p-value:** 6.23 × 10⁻³¹  

**Comment:**  
The estimated coefficient is close to the theoretical value of 0.4, which is the expected slope when regressing x on y without an intercept in this data-generating process.  

The t-statistic is very large and the p-value is essentially zero, so we strongly reject the null hypothesis `H0: β = 0`.  

This indicates that y is highly significant in explaining x when no intercept is included in the model.

### Q4(c)

The results from (a) and (b) are not simple reciprocals of each other.

In part (a), regressing y on x gives an estimated slope near 2, which matches the true data-generating process.  

In part (b), regressing x on y yields an estimated slope near 0.4, not 0.5, because the OLS formulas use different denominators:  
- **y on x**:  
  $$
  \hat{\beta} = \frac{\sum x_i y_i}{\sum x_i^2}
  $$  

- **x on y**:  
  $$
  \hat{\beta} = \frac{\sum x_i y_i}{\sum y_i^2}
  $$  

Thus, the two regression slopes are related but not inverses of one another. The difference arises from how OLS minimizes squared errors depending on which variable is treated as the response.


### Q4(d)

**Algebra derivation**  
For simple linear regression without an intercept  

$$
y_i = \beta x_i + \varepsilon_i
$$  

- The slope estimate is  

$$
\hat\beta = \frac{\sum x_i y_i}{\sum x_i^2}.
$$  

- The RSS is  

$$
\sum (y_i - \hat\beta x_i)^2 = \sum y_i^2 - \frac{(\sum x_i y_i)^2}{\sum x_i^2}.
$$  

- The standard error of beta_hat is  

$$
SE(\hat\beta) = \sqrt{\frac{\text{RSS}}{(n-1)\sum x_i^2}}.
$$  

Putting these together, the t-statistic is  

$$
t = \frac{\hat\beta}{SE(\hat\beta)} 
  = \frac{\sqrt{n-1}\,\sum x_i y_i}{\sqrt{\left(\sum x_i^2\right)\left(\sum y_i^2\right) - \left(\sum x_i y_i\right)^2}}.
$$


In [26]:
# Q4(d) Numerical verification 

import numpy as np
import pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(1)
x = rng.normal(size=100)
y = 2 * x + rng.normal(size=100)

X = pd.DataFrame({'x': x})        
fit = sm.OLS(y, X).fit()
t_statsmodels = fit.tvalues.iloc[0]  

Sxx = np.sum(x**2)
Syy = np.sum(y**2)
Sxy = np.sum(x*y)
n = len(x)

t_formula = (np.sqrt(n-1) * Sxy) / np.sqrt(Sxx*Syy - Sxy**2)

print("t (statsmodels):", t_statsmodels)
print("t (formula)    :", t_formula)
print("Difference     :", abs(t_statsmodels - t_formula))


t (statsmodels): 16.898417063035104
t (formula)    : 16.898417063035094
Difference     : 1.0658141036401503e-14


**Conclusion**  
The t-statistic from `statsmodels` and from the closed-form expression are numerically identical (up to rounding error).  
This confirms that the derived algebraic formula for the t-statistic in the no-intercept regression is correct.  

### Q4(e) 

The t-statistic is the same whether we regress y on x or x on y (without intercept).  
This is because the closed-form formula for the t-statistic is symmetric in x and y:  

$$
t = \frac{\sqrt{n-1}\,\sum x_i y_i}{\sqrt{(\sum x_i^2)(\sum y_i^2) - (\sum x_i y_i)^2}}
$$

Swapping x and y does not change either the numerator or the denominator.  
Therefore, both regressions yield the same t-value.  
Intuitively, this test is really about whether x and y are linearly related, and that relationship is symmetric.


In [27]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(1)
x = rng.normal(size=100)
y = 2 * x + rng.normal(size=100)
n = len(x)

# y on x WITH intercept 
X1 = sm.add_constant(pd.DataFrame({'x': x}))  # add intercept
fit_y_on_x = sm.OLS(y, X1).fit()
t_y_on_x = fit_y_on_x.tvalues.loc['x']  # t for slope of x

# x on y WITH intercept 
X2 = sm.add_constant(pd.DataFrame({'y': y}))  # add intercept
fit_x_on_y = sm.OLS(x, X2).fit()
t_x_on_y = fit_x_on_y.tvalues.loc['y']  # t for slope of y

#  compute t from correlation formula 
r = np.corrcoef(x, y)[0, 1]
t_from_r = r * np.sqrt((n - 2) / (1 - r**2))

print("t (y on x, with intercept):", t_y_on_x)
print("t (x on y, with intercept):", t_x_on_y)
print("t (from correlation r)    :", t_from_r)
print("Differences:")
print(" |t_y_on_x - t_x_on_y|    :", abs(t_y_on_x - t_x_on_y))
print(" |t_y_on_x - t_from_r|    :", abs(t_y_on_x - t_from_r))


t (y on x, with intercept): 16.734055202403045
t (x on y, with intercept): 16.734055202403038
t (from correlation r)    : 16.734055202403038
Differences:
 |t_y_on_x - t_x_on_y|    : 7.105427357601002e-15
 |t_y_on_x - t_from_r|    : 7.105427357601002e-15


### Q4(f)

In simple linear regression **with an intercept**, the t-statistic for testing  
H0 : beta_1 = 0 can be expressed as

$$
t = r \sqrt{\frac{n - 2}{1 - r^2}}
$$

\(r\) is the sample correlation between \(x\) and \(y\).

Since correlation is symmetric Corr(x,y) = Corr(y,x), the t-statistics are the same whether we regress \(y\) on \(x\) or \(x\) on \(y\).

**Numerical verification:**

Both regressions (with intercept) and the correlation formula give t is about 16.73, with differences smaller than 10^-14.  
This confirms the theoretical result.

